In [0]:
#TODO: save model, plot curves

In [5]:
!unzip /content/office.zip

Archive:  /content/office.zip
   creating: office/
   creating: office/webcam/
   creating: office/dslr/
   creating: office/amazon/
   creating: office/webcam/images/
   creating: office/dslr/images/
   creating: office/amazon/images/
   creating: office/webcam/images/trash_can/
  inflating: office/webcam/images/trash_can/frame_0021.jpg  
  inflating: office/webcam/images/trash_can/frame_0020.jpg  
  inflating: office/webcam/images/trash_can/frame_0019.jpg  
  inflating: office/webcam/images/trash_can/frame_0018.jpg  
  inflating: office/webcam/images/trash_can/frame_0017.jpg  
  inflating: office/webcam/images/trash_can/frame_0016.jpg  
  inflating: office/webcam/images/trash_can/frame_0015.jpg  
  inflating: office/webcam/images/trash_can/frame_0014.jpg  
  inflating: office/webcam/images/trash_can/frame_0013.jpg  
  inflating: office/webcam/images/trash_can/frame_0012.jpg  
  inflating: office/webcam/images/trash_can/frame_0011.jpg  
  inflating: office/webcam/images/trash_can/fram

In [0]:
# train.py
def train(model, optimizer, epoch, _lambda):
    """
    This method fits the network params one epoch at a time.
    Implementation based on:
    https://github.com/SSARCandy/DeepCORAL/blob/master/main.py
    """
    
    model.train()
    
    results = [] # list to append loss values at each epoch
    
    # first cast into an iterable list the data loaders
    # data_source: (batch_size, channels, height, width)
    # data_target: (batch_size)
    # source[0][1][0].size() --> torch.Size([128, 3, 224, 224])

    source, target = list(enumerate(source_loader)), list(enumerate(target_loader))
    train_steps = min(len(source), len(target))
    
    # start batch training
    for batch_idx in tnrange(train_steps):
        # fetch data in batches
        # _, source_data -> torch.Size([128, 3, 224, 224]), labels -> torch.Size([128])
        _, (source_data, source_label) = source[batch_idx]
        _, (target_data, _) = target[batch_idx] # unsupervised learning
        
        print("CUDA:", CUDA)
        if CUDA:
            # move to device
            source_data = source_data.cuda()
            source_label = source_label.cuda()
            target_data = target_data.cuda()
        
        # create pytorch variables, the variables and functions build a dynamic graph of computation
        source_data, source_label = Variable(source_data), Variable(source_label)
        target_data = Variable(target_data)
        
        # reset to zero optimizer gradients
        optimizer.zero_grad()
        
        # do a forward pass through network (recall DeepCORAL outputs source, target activation maps)
        output1, output2 = model(source_data, target_data)
        
        # compute losses (classification and coral loss)
        classification_loss = torch.nn.functional.cross_entropy(output1, source_label)
        coral_loss = CORAL_loss(output1, output2)
        
        print(type(coral_loss))

        # compute total loss (equation 6 paper)
        total_loss = classification_loss + _lambda*coral_loss
        
        # compute gradients of network (backprop in pytorch)
        total_loss.backward()
        
        # update weights of network
        optimizer.step()
        
        # append results for each batch iteration as dictionaries
        results.append({
            'epoch': epoch,
            'step': batch_idx + 1,
            'total_steps': train_steps,
            'lambda': _lambda,
            'coral_loss': coral_loss.item(), # coral_loss.data[0],
            'classification_loss': classification_loss.item(),  # classification_loss.data[0],
            'total_loss': total_loss.item() # total_loss.data[0]
        })
        
        # print training info
        print('Train Epoch: {:2d} [{:2d}/{:2d}]\t'
              'Lambda value: {:.4f}, Classification loss: {:.6f}, CORAL loss: {:.6f}, Total_Loss: {:.6f}'.format(
                  epoch,
                  batch_idx + 1,
                  train_steps,
                  _lambda,
                  classification_loss.item(), # classification_loss.data[0],
                  coral_loss.item(), # coral_loss.data[0],
                  total_loss.item() # total_loss.data[0]
              ))

    return results

# test.py
def test(model, data_loader, epoch):
    """
    Computes classification accuracy of (labeled) data using cross-entropy.
    Retreived from: https://github.com/SSARCandy/DeepCORAL/blob/master/main.py
    """
    # eval() it indicates the model that nothing new is 
    # to be learnt and the model is used for testing
    model.eval()
    
    test_loss = 0
    correct_class = 0
    
    # go over dataloader batches, labels
    for data, label in data_loader:
        if CUDA:
            data, label = data.cuda(), label.cuda()
        
        # note on volatile: https://stackoverflow.com/questions/49837638/what-is-volatile-variable-in-pytorch
        data, label = Variable(data, volatile=True), Variable(label)
        output, _ = model(data, data) # just use one ouput of DeepCORAL
        
        # sum batch loss when computing classification
        test_loss += torch.nn.functional.cross_entropy(output, label, size_average=False).item()
        # test_loss += torch.nn.functional.cross_entropy(output, label, size_average=False).data[0]
        
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct_class += pred.eq(label.data.view_as(pred)).cpu().sum()
        
    
    # compute test loss as correclty classified labels divided by total data size
    test_loss = test_loss/len(data_loader.dataset)
    
    # return dictionary containing info of each epoch
    return {
        "epoch": epoch,
        "average_loss": test_loss,
        "correct_class": correct_class,
        "total_elems": len(data_loader.dataset),
        "accuracy %": 100.*correct_class/len(data_loader.dataset)
    }

In [0]:
# main.py
from __future__ import division
import argparse
import warnings
from tqdm import tnrange
warnings.filterwarnings("ignore")

import torch
from torch.autograd import Variable

from loss import CORAL_loss
from utils import load_pretrained_AlexNet, save_model, load_model
from dataloader import get_office_dataloader
from model import DeepCORAL, AlexNet

In [3]:
# define train parameters as in the paper (page 5)
CUDA = True if torch.cuda.is_available() else False
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9
BATCH_SIZE = [32, 32] # batch_s, batch_t [128, 56]
EPOCHS = 1

# create dataloaders (Amazon as source and Webcam as target)
# TODO: try different combinations between 3 datasets to test adaptation
source_loader = get_office_dataloader(name_dataset="amazon", batch_size=BATCH_SIZE[0])
target_loader = get_office_dataloader(name_dataset="webcam", batch_size=BATCH_SIZE[1])

# argparse ...
model = DeepCORAL(num_classes=31) # input no. of classes in custom dataset

# define optimizer pytorch: https://pytorch.org/docs/stable/optim.html
# specify per-layer learning rates: 10*learning_rate for last two fc layers according to paper
optimizer = torch.optim.SGD([
    {"params": model.sharedNetwork.parameters()},
    {"params": model.fc8.parameters(), "lr":10*LEARNING_RATE},
], lr=LEARNING_RATE, momentum=MOMENTUM)

if CUDA:
    model = model.cuda()
    print("using cuda...")

using cuda...


In [4]:
# load pre-trained model --> TODO: check it is loading properly
# if args.load is not NONE:
    # load_model(model, args.load)
# else:
    # load_pretrained_AlexNet(model.sharedNetwork, progress=True)

load_pretrained_AlexNet(model.sharedNetwork, progress=True)

loading pre-trained model...
loaded model correctly...


In [5]:
# store statistics of train/test
training_s_statistic = []
testing_s_statistic = []
testing_t_statistic = []

# iterate over epochs
for epoch in tnrange(EPOCHS):
    # compute lambda value
    _lambda = (epoch+1)/EPOCHS
    
    result_train = train(model, optimizer, epoch+1, _lambda)
    
    print('###EPOCH {}: Classification: {:.6f}, CORAL loss: {:.6f}, Total_Loss: {:.6f}'.format(
            epoch+1,
            sum(row['classification_loss'] / row['total_steps'] for row in result_train),
            sum(row['coral_loss'] / row['total_steps'] for row in result_train),
            sum(row['total_loss'] / row['total_steps'] for row in result_train),
        ))
    
    training_s_statistic.append(result_train)
    
    # perform testing simultaneously: classification accuracy on both dataset
    test_source = test(model, source_loader, epoch)
    test_target = test(model, target_loader, epoch)
    testing_s_statistic.append(test_source)
    testing_t_statistic.append(test_target)
    
    print('###Test Source: Epoch: {}, avg_loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
            epoch+1,
            test_source['average_loss'],
            test_source['correct_class'],
            test_source['total_elems'],
            test_source['accuracy %'],
        ))

    print('###Test Target: Epoch: {}, avg_loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
            epoch+1,
            test_target['average_loss'],
            test_target['correct_class'],
            test_target['total_elems'],
            test_target['accuracy %'],
    ))

CUDA: True
<class 'torch.Tensor'>
Train Epoch:  1 [ 1/25]	Lambda value: 1.0000, Classification loss: 3.496027, CORAL loss: 0.000016, Total_Loss: 3.496044
CUDA: True
<class 'torch.Tensor'>
Train Epoch:  1 [ 2/25]	Lambda value: 1.0000, Classification loss: 3.669545, CORAL loss: 0.000053, Total_Loss: 3.669597
CUDA: True
<class 'torch.Tensor'>
Train Epoch:  1 [ 3/25]	Lambda value: 1.0000, Classification loss: 2.849719, CORAL loss: 0.000215, Total_Loss: 2.849933
CUDA: True
<class 'torch.Tensor'>
Train Epoch:  1 [ 4/25]	Lambda value: 1.0000, Classification loss: 2.555252, CORAL loss: 0.001386, Total_Loss: 2.556638
CUDA: True
<class 'torch.Tensor'>
Train Epoch:  1 [ 5/25]	Lambda value: 1.0000, Classification loss: 2.753706, CORAL loss: 0.007681, Total_Loss: 2.761387
CUDA: True
<class 'torch.Tensor'>
Train Epoch:  1 [ 6/25]	Lambda value: 1.0000, Classification loss: 2.046711, CORAL loss: 0.017798, Total_Loss: 2.064508
CUDA: True
<class 'torch.Tensor'>
Train Epoch:  1 [ 7/25]	Lambda value: 1.00